In [16]:
from flask import Flask, render_template, request
import numpy as np
from keras.models import load_model
from keras.preprocessing import sequence
import pickle
import re
from pyvi import ViTokenizer

app = Flask(__name__)

# Load stopwords
STOPWORDS = './vietnamese-stopwords-dash.txt'
with open(STOPWORDS, "r", encoding='utf-8') as ins:
    stopwords = set([line.strip() for line in ins])

# Load the model
MODEL_FILE = './Text_CNN_model.h5'
model = load_model(MODEL_FILE)

# Load the tokenizer
with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Function to preprocess text
def filter_stop_words(text, stop_words):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def preprocess(text):
    text = ViTokenizer.tokenize(text)
    text = filter_stop_words(text, stopwords)
    text = deEmojify(text)
    text = text.lower()
    return text

emotion_labels = {
    0: "clean",
    1: "offensive",
    2: "hate"
}

# Function to predict sentiment
def TextPredict(txt):
    # Preprocess the input sentence
    input_sentence = preprocess(txt)

    # Tokenize and pad the input sentence
    input_sequence = tokenizer.texts_to_sequences([input_sentence])
    input_sequence = sequence.pad_sequences(input_sequence, maxlen=100)

    # Make the prediction
    predictions = model.predict(input_sequence)
    predicted_class = np.argmax(predictions)

    predicted_label = emotion_labels[predicted_class]
    return predicted_label

# Route for home page
@app.route('/')
def home():
    return render_template('index.html')

# Route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    comment = request.form['comment']
    predicted_class = TextPredict(comment)
    return render_template('index.html', comment=comment, prediction=predicted_class)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Jun/2024 21:34:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2024 21:34:27] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2024 21:34:27] "GET /static/image.jpg HTTP/1.1" 304 -


1/1 [==============================] - 0s 244ms/step


127.0.0.1 - - [16/Jun/2024 21:34:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2024 21:34:36] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Jun/2024 21:34:36] "GET /static/image.jpg HTTP/1.1" 304 -


1/1 [==============================] - 0s 103ms/step


127.0.0.1 - - [16/Jun/2024 21:34:50] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2024 21:34:51] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Jun/2024 21:34:51] "GET /static/image.jpg HTTP/1.1" 304 -
127.0.0.1 - - [16/Jun/2024 21:38:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2024 21:38:17] "GET /static/image.jpg HTTP/1.1" 304 -
127.0.0.1 - - [16/Jun/2024 21:38:17] "GET /static/style.css HTTP/1.1" 304 -


1/1 [==============================] - 0s 44ms/step


127.0.0.1 - - [16/Jun/2024 21:38:21] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2024 21:38:22] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Jun/2024 21:38:22] "GET /static/image.jpg HTTP/1.1" 304 -


In [3]:
%tb

SystemExit: 1